In [28]:
import os 
import pandas as pd 
import numpy as np 

In [253]:
year = 'after1y'
followup = '2_year_follow_up_y_arm_1' #options = ['baseline_year_1_arm_1', '1_year_follow_up_y_arm_1', '2_year_follow_up_y_arm_1']

cat_cov = ['sex', 'abcd.site', 'race.ethnicity'] 
num_cov = ['age', 'BMI_sds_baseline', 'high.educ', 'income'] 

In [254]:
original_phenotype_dir = '/Users/wangheehwan/Desktop/CNN_for_BMI/paper/data/statistics/ABCD Release4.0 Tabular dataset.csv'
image_subject_list_dir = '/Users/wangheehwan/Desktop/CNN_for_BMI/paper/data/dnn/pretraining_subject_list.csv'
if year == 'after1y':
    phenotype_dir  = '/Users/wangheehwan/Desktop/CNN_for_BMI/paper/data/dnn/ABCD_phenotype_total_1years_become_overweight_10PS_stratified_partitioned_5fold.csv'
    extract_list = ['subjectkey', 'BMI_sds_baseline', 'BMI_sds_1year','become_overweight', 'partition0', 'partition1', 'partition2', 'partition3', 'partition4']
elif year == 'after2y': 
    phenotype_dir  = '/Users/wangheehwan/Desktop/CNN_for_BMI/paper/data/dnn/ABCD_phenotype_total_2years_become_overweight_10PS_stratified_partitioned_5fold.csv'
    extract_list = ['subjectkey', 'BMI_sds_baseline', 'BMI_sds_2year','become_overweight', 'partition0', 'partition1', 'partition2', 'partition3', 'partition4']

image_subject_list = pd.read_csv(image_subject_list_dir)['subjectkey']
phenotype = pd.read_csv(phenotype_dir)
phenotype = pd.merge(phenotype, image_subject_list, how='inner', on='subjectkey')
original_phenotype = pd.read_csv(original_phenotype_dir)
original_phenotype = original_phenotype[original_phenotype['eventname'] == followup].reset_index()
for i in range(len(original_phenotype)): 
    original_phenotype['subjectkey'][i] = original_phenotype['subjectkey'][i].replace("_", "")
phenotype_final_tmp = pd.merge(original_phenotype, phenotype[extract_list], how='inner', on='subjectkey')
phenotype_final_tmp = phenotype_final_tmp[phenotype_final_tmp['become_overweight'] == 1]

phenotype = pd.DataFrame()
for i in range(5): 
    phenotype = pd.concat([phenotype,  phenotype_final_tmp[phenotype_final_tmp['partition%s' % i] == 'test']])
phenotype = phenotype.drop_duplicates()


/var/folders/_6/fts0bv3j7jjb0263g94q1lbw0000gn/T/ipykernel_46259/638377041.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  original_phenotype['subjectkey'][i] = original_phenotype['subjectkey'][i].replace("_", "")


## sex

In [231]:
index, count = pd.value_counts(phenotype['sex']).index,  pd.value_counts(phenotype['sex']).values
index, count = np.sort(index), count[np.argsort(index)]
print(index)
print(count)
print(count[1] / np.sum(count) * 100)
    
    


['F' 'M']
[224 249]
52.64270613107822


## BMI sds

In [255]:
if followup == 'baseline_year_1_arm_1':
    print(np.median(phenotype['BMI_sds_baseline']))
elif followup == '1_year_follow_up_y_arm_1': 
    if year == 'after1y': 
        print(np.median(phenotype['BMI_sds_1year']))
    elif year == 'after2y':
        tmp = pd.read_csv('/Users/wangheehwan/Desktop/CNN_for_BMI/paper/data/dnn/ABCD_phenotype_total_1years_become_overweight_10PS_stratified_partitioned_5fold.csv')
        phenotype_tmp = pd.merge(phenotype, tmp[['subjectkey', 'BMI_sds_1year']], how='inner', on='subjectkey')
        print(np.median(phenotype_tmp['BMI_sds_1year']))
elif followup == '2_year_follow_up_y_arm_1': 
    if year == 'after1y': 
        tmp = pd.read_csv('/Users/wangheehwan/Desktop/CNN_for_BMI/paper/data/dnn/ABCD_phenotype_total_2years_become_overweight_10PS_stratified_partitioned_5fold.csv')
        phenotype_tmp = pd.merge(phenotype, tmp[['subjectkey', 'BMI_sds_2year']], how='inner', on='subjectkey')
        print(np.median(phenotype_tmp['BMI_sds_2year']))
    elif year == 'after2y':
        print(np.median(phenotype['BMI_sds_2year']))

1.20098287083091


## Income

In [233]:
print(np.median(phenotype['income'].dropna()))

8.0


## Maternal Education

In [250]:
print(np.median(phenotype['high_educ'].dropna()))

18.0


## Genetic Ancestry 

In [252]:
index, count = pd.value_counts(phenotype['euro']).index,  pd.value_counts(phenotype['euro']).values
index, count = np.sort(index), count[np.argsort(index)]
print(index)
print(count)
print(count[1] / np.sum(count) * 100)
    
    


[0. 1.]
[ 90 298]
76.80412371134021
